In [1]:
from bs4 import BeautifulSoup
import requests
from datetime import datetime, timedelta
import pandas as pd
from collections import defaultdict
import re

In [2]:
def hashnode():
    url = 'https://hashnode.com/community?source=header'
    html_content = requests.get(url)
    soup = BeautifulSoup(html_content.text, 'lxml')
    container = soup.find('div', attrs={'data-state':'active'})
    return container

In [3]:
def hashnode_data_points(html_body):
    all_articles = []
    
    for cont in html_body.find('div'):
        articles = cont.find('div', {'class':'flex flex-col gap-1 '})
    
    #extract article titles
        if cont.find('div', {'class':'flex flex-col gap-1'}) != None:
            article_titles = cont.find('h1').text

        #extract article authors name
        if cont.find('div', {'class': 'flex flex-row items-center justify-start gap-3'}) != None:
            names = cont.find('span', {'class':'font-semibold text-slate-700 dark:text-slate-200 cursor-pointer'}).text


        if cont.find('div', {'class': 'flex flex-row items-center justify-start gap-3'}) != None:
            dates = cont.find('p', {'class':'text-sm text-slate-500 dark:text-slate-400 font-normal'}).text


        #extracting the tag holding the article links    
        if cont.find('div', {'class': 'flex flex-row items-center justify-start gap-1'}) != None:
            article_links = cont.find('div', {'class': 'flex flex-row items-center justify-start gap-1'}).find_all('a')
            if len(article_links) > 1:
                second_links = article_links[1]['href']


        #extracting the tag holding all likes and reads of the article
        if cont.find('section', {'class':'flex flex-col gap-5'}) != None:
            reactions = cont.find('section', {'class':'flex flex-col gap-5'}).find_all('p')
            if len(reactions) > 3:
                likes = reactions[1].text
                reads = reactions[3].text


        if cont.find('div', {'class':'hidden md:block'}) != None:
            article_contents = cont.find('span', {'class':'text-base hidden font-normal text-slate-500 dark:text-slate-400 hn-break-words cursor-pointer md:line-clamp-3'})            

    #append all extracted tags object 
        all_articles.append([article_titles, names, dates, second_links, likes, reads])
    return all_articles
    
        

In [4]:
hashnode_result = hashnode()

In [5]:
articles_dict_list=defaultdict(list)
articles_colnames = ['Title', 'AuthorName', 'DatePosted', 'ArticleLink', 'Likes', 'Reads']

In [6]:
articles_df = pd.DataFrame(hashnode_data_points(hashnode_result), columns=articles_colnames)

In [7]:
def article_content(links):
    request_link = requests.get(links)
    soup_content = BeautifulSoup(request_link.text, 'lxml')
    content = soup_content.find('div', {'id':'post-content-wrapper'}).text
    return content

articles_df['ArticleContent'] = articles_df.apply(lambda row: article_content(row['ArticleLink']), axis=1)

articles_df['ArticleContent'] = articles_df['ArticleContent'].replace(r'^\n', '', regex=True)
articles_df['ArticleContent'] = articles_df['ArticleContent'].replace(r'\n', ' ', regex=True)

In [8]:

def convert_to_date(value):
    try:
        return pd.to_datetime(value).date()
    except ValueError:
        return datetime.today().date()
    
articles_df['DatePosted'] = articles_df['DatePosted'].apply(convert_to_date)

In [9]:
articles_df

,Title,AuthorName,DatePosted,ArticleLink,Likes,Reads,ArticleContent
0,Setting Up Post Schedules with EventBridge Sch...,Sandro Volpicella,2024-01-16,https://engineering.hashnode.com/setting-up-po...,95 likes,819 reads,One essential feature of any blogging platform...
1,Rust 🦀 Hello world,Francesco Ciulla,2024-01-10,https://blog.francescociulla.com/rust-hello-world,110 likes,956 reads,Rust is a systems programming language that ru...
2,Rust 🦀 Hello world,Francesco Ciulla,2024-01-10,https://blog.francescociulla.com/rust-hello-world,110 likes,956 reads,Rust is a systems programming language that ru...
3,React Design Patterns ⭐,Nishit Bariya,2024-01-19,https://nishithacks.hashnode.dev/react-design-...,32 likes,33 reads,In this article you’ll learn How React Develop...
4,SliceIt URL Shortener : API Documentation,Tenzopy Digital,2024-01-19,https://blog.tenzopy.com/sliceitapi,32 likes,33 reads,A URL Shortener Made on Django Framework with ...
5,Building UniFocus: A Colorful To-Do List App w...,Cindy Kandie,2024-01-19,https://cindykandie.hashnode.dev/building-unif...,32 likes,33 reads,"Hey there, fellow developers and good vibe ent..."
6,Building UniFocus: A Colorful To-Do List App w...,Cindy Kandie,2024-01-19,https://cindykandie.hashnode.dev/building-unif...,32 likes,33 reads,"Hey there, fellow developers and good vibe ent..."
7,Are you into Gbeborun?,Mercy Kalu,2024-01-19,https://lettersfromkath.com/are-you-into-gbegb...,32 likes,33 reads,"September 2023, Lagos I met a tall, dark-skinn..."
8,Day 5 of 100days of code,chioma christopher,2024-01-19,https://kaylah.hashnode.dev/day-5-of-100days-o...,32 likes,33 reads,"Recap : Yesterday, I created a Steps component..."
9,Day 5 of 100days of code,chioma christopher,2024-01-19,https://kaylah.hashnode.dev/day-5-of-100days-o...,32 likes,33 reads,"Recap : Yesterday, I created a Steps component..."
